<a href="https://colab.research.google.com/github/Crawnicles/Algo-trading-project/blob/main/VolatilityBreakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project setup
Configure imports and setup the connection to the Alpaca API

In [ ]:
!pip install python_dotenv
!pip install alpaca_trade_api
!pip install backtrader

In [4]:
from alpaca_trade_api.rest import TimeFrame
import alpaca_trade_api as tradeapi
from alpaca_trade_api.stream import Stream
import backtrader as bt
import matplotlib
import pandas as pd
from alpaca_trade_api import TimeFrameUnit
from pandas.core.frame import DataFrame
import numpy as np

In [5]:
# Alpaca API info
ALPACA_API_KEY = 'PKZ7UQDMAWO5UAULDXCO'
ALPACA_SECRET_KEY = 'ESBhzVmXVPtUjbzdmKx3airlPQ5eBHlbamnNy123'

rest_api = tradeapi.REST(ALPACA_API_KEY,
                ALPACA_SECRET_KEY, 
                'https://paper-api.alpaca.markets',
                'v2')

# Configure Base algorithim

# Gather Data Frames

In [12]:
def VolBreakout(PD_Dataframe, percentage: int):
  # Convert int to Percentage
  multiplier = percentage / 100

  # Calculate the Top and Bottom of the range to determine where positions will be taken
  top_range_calc = PD_Dataframe['high'][-1] * multiplier
  bottom_range_calc = PD_Dataframe['low'][-1] * multiplier
  top_of_range = PD_Dataframe['open'] + top_range_calc
  bottom_of_range = PD_Dataframe['open'] - bottom_range_calc

  # Initiate a Column that counts the number of times a cross happens on above the range and below the range
  PD_Dataframe['Above Range'] = 0.0
  PD_Dataframe['Below Range'] = 0.0

  # Find the columns where a cross happens and updates the column with a 1 if there is a cross
  PD_Dataframe['Above Range'] = np.where(PD_Dataframe['close'] > top_of_range, 1, 0)
  PD_Dataframe['Below Range'] = np.where(PD_Dataframe['close'] < bottom_of_range, 1, 0)

  # Creates two columns to indicate a Long or Short Signal
  PD_Dataframe['Long Signal'] = PD_Dataframe['Above Range'].diff()
  PD_Dataframe['Short Signal'] = PD_Dataframe['Below Range'].diff()

  return(PD_Dataframe)

In [13]:
spy_bars = rest_api.get_bars('TSLA', TimeFrame.Day, '2020-01-01', '2020-12-31').df
spy_bars

,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,
2020-01-02 05:00:00+00:00,424.50,430.6957,421.7100,430.26,9558410,136850,427.110933
2020-01-03 05:00:00+00:00,440.01,454.0000,436.9200,443.01,17801880,236698,445.732623
2020-01-06 05:00:00+00:00,440.35,451.5600,440.0000,451.50,10174685,134223,445.923763
2020-01-07 05:00:00+00:00,461.40,471.6300,453.3550,469.06,18214838,250188,463.832116
2020-01-08 05:00:00+00:00,474.00,498.4900,468.2300,492.14,31235728,413649,485.693288
...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,643.01,666.0900,641.0000,661.77,22865568,465757,658.508022
2020-12-28 05:00:00+00:00,674.68,681.4000,660.8000,663.69,32278861,627137,669.639927
2020-12-29 05:00:00+00:00,660.86,669.9000,655.0000,665.99,22910816,465753,662.650078


In [14]:
VolBreakout(spy_bars, 1)

,open,high,low,close,volume,trade_count,vwap,Above Range,Below Range,Signal,Long Signal,Short Signal
timestamp,,,,,,,,,,,,
2020-01-02 05:00:00+00:00,424.50,430.6957,421.7100,430.26,9558410,136850,427.110933,0.0,0.0,,NaN,NaN
2020-01-03 05:00:00+00:00,440.01,454.0000,436.9200,443.01,17801880,236698,445.732623,0.0,0.0,,0.0,0.0
2020-01-06 05:00:00+00:00,440.35,451.5600,440.0000,451.50,10174685,134223,445.923763,1.0,0.0,,1.0,0.0
2020-01-07 05:00:00+00:00,461.40,471.6300,453.3550,469.06,18214838,250188,463.832116,1.0,0.0,,0.0,0.0
2020-01-08 05:00:00+00:00,474.00,498.4900,468.2300,492.14,31235728,413649,485.693288,1.0,0.0,,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24 05:00:00+00:00,643.01,666.0900,641.0000,661.77,22865568,465757,658.508022,1.0,0.0,,0.0,0.0
2020-12-28 05:00:00+00:00,674.68,681.4000,660.8000,663.69,32278861,627137,669.639927,0.0,1.0,,-1.0,1.0
2020-12-29 05:00:00+00:00,660.86,669.9000,655.0000,665.99,22910816,465753,662.650078,0.0,0.0,,0.0,-1.0


In [39]:
# Creates a Function to perform backtesting using the alpaca API
def run_backtest(strategy, symbols, start, end, timeframe=TimeFrame.Day, cash=10000):
	"""params:
          strategy: the strategy you wish to backtest, an instance of backtrader.Strategy
          symbols: the symbol (str) or list of symbols List[str] you wish to backtest on
          start: start date of backtest in format 'YYYY-MM-DD'
          end: end date of backtest in format: 'YYYY-MM-DD'
          timeframe: the timeframe the strategy trades on (size of bars) -
                      1 min: TimeFrame.Minute, 1 day: TimeFrame.Day, 5 min: TimeFrame(5, TimeFrameUnit.Minute)
          cash: the starting cash of backtest
    """

	# initialize backtrader broker
	cerebro = bt.Cerebro(stdstats=True)
	cerebro.broker.setcash(cash)

	# add strategy
	cerebro.addstrategy(strategy)

	# add analytics
	# cerebro.addobserver(bt.observers.Value)
	# cerebro.addobserver(bt.observers.BuySell)
	cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mysharpe')
	# historical data request
	if type(symbols) == str:
		symbol = symbols
		alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
		data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
		cerebro.adddata(data)
	elif type(symbols) == list or type(symbols) == set:
		for symbol in symbols:
			alpaca_data = rest_api.get_bars(symbol, timeframe, start, end, adjustment='all').df
			data = bt.feeds.PandasData(dataname=alpaca_data, name=symbol)
			cerebro.adddata(data)


	# run
	initial_portfolio_value = cerebro.broker.getvalue()
	print(f'Starting Portfolio Value: {initial_portfolio_value}')
	results = cerebro.run()
	final_portfolio_value = cerebro.broker.getvalue()
	print(f'Final Portfolio Value: {final_portfolio_value} ---> Return: {(final_portfolio_value/initial_portfolio_value - 1)*100}%')

	strat = results[0]
	print('Sharpe Ratio:', strat.analyzers.mysharpe.get_analysis()['sharperatio'])
	cerebro.plot(iplot= False)